# Easy-OCR 

In [21]:
import os
import pandas as pd
from pathlib import Path
import torch
import easyocr
import traceback
from tqdm import tqdm
import gc

# ========== 初始化 OCR 模型 ==========
reader = easyocr.Reader(['en'], gpu=True)

# ========== 路徑設定 ==========
img_dir = Path("/home/ne6131039/Desktop/TEM_DATAS/LLaVA Dataset/TEM_descriptions")
csv_path = Path("easyocr_output.csv")
error_csv_path = Path("easyocr_errors.csv")
image_extensions = {'.png', '.jpg', '.jpeg', '.tif', '.tiff'}

# ========== 初始化 CSV（若不存在） ==========
if not csv_path.exists() or csv_path.stat().st_size == 0:
    pd.DataFrame(columns=['image_name','easyocr_ocr']).to_csv(csv_path, index=False)

# ========== 載入已處理紀錄 ==========
df_existing = pd.read_csv(csv_path)
done_set = set(df_existing["image_name"])

# # ========== 熱機（避免首次執行太慢） ==========
sample_img = "/home/ne6131039/Desktop/TEM_DATAS/LLaVA Dataset/TEM_descriptions/PDF2_Image3.png"
_ = reader.readtext(str(sample_img), detail=0)
print("[INFO] Warm-up complete.")

# ========== 圖片列表 ==========
all_imgs = sorted([p for p in img_dir.iterdir() if p.suffix.lower() in image_extensions])
records = []
error_log = []

# ========== 開始處理 ==========
BATCH_SAVE = 100
counter = 0

for img_path in tqdm(all_imgs, desc="Processing"):
    img_name = img_path.name
    if img_name in done_set:
        continue

    try:
        easy_texts = reader.readtext(str(img_path), detail=0)
        easyocr_ocr = " ".join(easy_texts)
    except Exception as e:
        easyocr_ocr = f"[ERROR: EasyOCR] {e}\n{traceback.format_exc(limit=1)}"
        error_log.append({"image": img_name, "module": "EasyOCR", "error": easyocr_ocr})

    records.append({
        "image_name": img_name,
        "easyocr_ocr": easyocr_ocr
    })

    counter += 1

    # ==== 批次儲存新資料 ====
    if counter % BATCH_SAVE == 0:
        df_batch = pd.DataFrame(records)
        df_batch.to_csv(csv_path, mode='a', header=False, index=False)
        records.clear()
        print(f"已新增 {BATCH_SAVE} 筆資料至 {csv_path}")
        torch.cuda.empty_cache()
        gc.collect()

# ========== 最後儲存 ==========
if records:
    pd.DataFrame(records).to_csv(csv_path, mode='a', header=False, index=False)
    print(f"最後新增 {len(records)} 筆資料至 {csv_path}")

# ========== 儲存錯誤 log ==========
if error_log:
    pd.DataFrame(error_log).to_csv(error_csv_path, index=False)
    print(f"錯誤紀錄已儲存至 {error_csv_path}")
else:
    print("全部圖片處理成功，無錯誤。")

print(f"全部處理完成！")


[INFO] Warm-up complete.


Processing:  21%|████▉                   | 10096/48694 [00:17<01:33, 413.93it/s]

已新增 100 筆資料至 easyocr_output.csv


Processing:  21%|█████                   | 10180/48694 [00:37<05:05, 125.93it/s]

已新增 100 筆資料至 easyocr_output.csv


...

已新增 100 筆資料至 easyocr_output.csv


Processing: 100%|██████████████████████▉| 48600/48694 [3:03:44<00:34,  2.76it/s]

已新增 100 筆資料至 easyocr_output.csv


Processing: 100%|███████████████████████| 48694/48694 [3:04:07<00:00,  4.41it/s]

最後新增 94 筆資料至 easyocr_output.csv
全部圖片處理成功，無錯誤。
全部處理完成！


# Pix2Text

In [1]:
import os
import pandas as pd
from pathlib import Path
from pix2text import Pix2Text
import torch
import traceback
from tqdm import tqdm
import gc

# ========== 初始化 OCR 模型 ==========
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"[INFO] Pix2Text is using device: {device}")
ocr_agent = Pix2Text(det_model='mfd', device=device)

# ========== 路徑設定 ==========
img_dir = Path("/home/ne6131039/Desktop/TEM_DATAS/LLaVA Dataset/TEM_descriptions")
csv_path = Path("p2tocr_output.csv")
error_csv_path = Path("p2tocr_errors.csv")
image_extensions = {'.png', '.jpg', '.jpeg', '.tif', '.tiff'}

# ========== 初始化 CSV（若不存在） ==========
if not csv_path.exists() or csv_path.stat().st_size == 0:
    pd.DataFrame(columns=['image_name', 'pix2text_ocr']).to_csv(csv_path, index=False)

# ========== 載入已處理紀錄 ==========
df_existing = pd.read_csv(csv_path)
done_set = set(df_existing["image_name"])

# # ========== 熱機（避免首次執行太慢） ==========
sample_img = "/home/ne6131039/Desktop/TEM_DATAS/LLaVA Dataset/TEM_descriptions/PDF2_Image3.png"
_ = ocr_agent.recognize(str(sample_img))
print("[INFO] Warm-up complete.")

# ========== 圖片列表 ==========
all_imgs = sorted([p for p in img_dir.iterdir() if p.suffix.lower() in image_extensions])
records = []
error_log = []

# ========== 開始處理 ==========
BATCH_SAVE = 100
counter = 0

for img_path in tqdm(all_imgs, desc="Processing"):
    img_name = img_path.name
    if img_name in done_set:
        continue

    try:
        pix2text_ocr = ocr_agent.recognize(str(img_path))
    except Exception as e:
        pix2text_ocr = f"[ERROR: Pix2Text] {e}\n{traceback.format_exc(limit=1)}"
        error_log.append({"image": img_name, "module": "Pix2Text", "error": pix2text_ocr})

    records.append({
        "image_name": img_name,
        "pix2text_ocr": pix2text_ocr,
    })

    counter += 1

    # ==== 批次儲存新資料 ====
    if counter % BATCH_SAVE == 0:
        df_batch = pd.DataFrame(records)
        df_batch.to_csv(csv_path, mode='a', header=False, index=False)
        records.clear()
        print(f"已新增 {BATCH_SAVE} 筆資料至 {csv_path}")
        torch.cuda.empty_cache()
        gc.collect()

# ========== 最後儲存 ==========
if records:
    pd.DataFrame(records).to_csv(csv_path, mode='a', header=False, index=False)
    print(f"最後新增 {len(records)} 筆資料至 {csv_path}")

# ========== 儲存錯誤 log ==========
if error_log:
    pd.DataFrame(error_log).to_csv(error_csv_path, index=False)
    print(f"錯誤紀錄已儲存至 {error_csv_path}")
else:
    print("全部圖片處理成功，無錯誤。")

print(f"全部處理完成！")


2025-06-16 13:58:04.602370701 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2025-06-16 13:58:04.602390158 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
2025-06-16 13:58:04.639163480 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load li

[INFO] Pix2Text is using device: cuda


2025-06-16 13:58:04.741286786 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2025-06-16 13:58:04.741299406 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
2025-06-16 13:58:04.846274356 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load li

Loading /home/ne6131039/.pix2text/1.1/mfd-onnx/mfd-v20240618.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider

0: 96x768 1 embedding, 8.9ms
Speed: 7.8ms preprocess, 8.9ms inference, 63.2ms postprocess per image at shape (1, 3, 96, 768)


2025-06-16 13:58:05.421689446 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2025-06-16 13:58:05.421702960 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  8.37it/s]


...


WARNING ⚠️ imgsz=[81, 768] must be multiple of max stride 32, updating to [96, 768]
0: 96x768 1 embedding, 10.5ms
Speed: 0.4ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 768)



Processing: 100%|████████████████████████▉| 48693/48694 [44:10<00:01,  1.36s/it]


WARNING ⚠️ imgsz=[83, 768] must be multiple of max stride 32, updating to [96, 768]
0: 96x768 4 embeddings, 12.1ms
Speed: 0.5ms preprocess, 12.1ms inference, 0.9ms postprocess per image at shape (1, 3, 96, 768)



Processing: 100%|█████████████████████████| 48694/48694 [44:12<00:00, 18.36it/s]

最後新增 94 筆資料至 p2tocr_output.csv
全部圖片處理成功，無錯誤。
全部處理完成！


In [4]:
import os
import pandas as pd
from pathlib import Path
from pix2text import Pix2Text
import torch
import traceback
from tqdm import tqdm
import gc
# ========== 初始化 OCR 模型 ==========
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"[INFO] Pix2Text is using device: {device}")
ocr_agent = Pix2Text(det_model='mfd', device=device)

sample_img = "/home/ne6131039/Desktop/TEM_DATAS/LLaVA Dataset/TEM_descriptions/PDF2_Image3.png"
_ = ocr_agent.recognize(str(sample_img))
print("[INFO] Warm-up complete.")
print(_)


2025-06-15 16:44:32.023823508 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2025-06-15 16:44:32.023835378 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
2025-06-15 16:44:32,112 - OrtInferSession - WARNING: CUDAExecutionProvider is not avaiable for current env, the inference part is automatically shifted to be executed under CPUExecutionProvider.
2025-06-15 16:44:32.050949717 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetPr

[INFO] Pix2Text is using device: cuda


2025-06-15 16:44:32.149474550 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2025-06-15 16:44:32.149485824 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
2025-06-15 16:44:32.257710886 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load li

Loading /home/ne6131039/.pix2text/1.1/mfd-onnx/mfd-v20240618.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider



2025-06-15 16:44:32.632295587 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.11: cannot open shared object file: No such file or directory

2025-06-15 16:44:32.632306734 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:747 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.


0: 96x768 1 embedding, 12.6ms
Speed: 1.0ms preprocess, 12.6ms inference, 0.7ms postprocess per image at shape (1, 3, 96, 768)


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  9.60it/s]


[INFO] Warm-up complete.
Fig. 3 Sequential modification of two individual surface oxidised Pt-Ti nanoparticles using a focused electron beam. Initial state is an amor-
 $\mathrm{P t-T i}$ indicated by red circles in (a) and (b), however the interactions with the electron beam at a 30 pA current create a single Pt core phous alloy
indicated by a blue circle in (b) and (c). The time direction is from left to right. Red circles indicate the nanoparticle prior to modification and blue circles indicate the same nanoparticle post modification.



# combine two csv files 

In [4]:
import pandas as pd

# 讀取兩份 CSV
df1 = pd.read_csv("easyocr_output.csv")     # 包含 image_name, easyocr_ocr
df2 = pd.read_csv("p2tocr_output.csv")    # 包含 image_name, pix2text_ocr

# 先找出共同的 image_name（交集）
common_images = set(df1["image_name"]) & set(df2["image_name"])

# 分別篩選只包含交集的行
df1_filtered = df1[df1["image_name"].isin(common_images)]
df2_filtered = df2[df2["image_name"].isin(common_images)]

# 合併兩份資料
merged_df = pd.merge(df1_filtered, df2_filtered, on="image_name", how="inner")

# 儲存結果
merged_df.to_csv("ocr_output.csv", index=False)
print(f"✅ 已合併 {len(merged_df)} 筆，僅保留同時有兩種 OCR 結果的圖片。")


✅ 已合併 48694 筆，僅保留同時有兩種 OCR 結果的圖片。
